In [ ]:
# Copyright 2019 Google Inc.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#     http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

#Predicting Movie Review Sentiment with BERT on TF Hub

If you’ve been following Natural Language Processing over the past year, you’ve probably heard of BERT: Bidirectional Encoder Representations from Transformers. It’s a neural network architecture designed by Google researchers that’s totally transformed what’s state-of-the-art for NLP tasks, like text classification, translation, summarization, and question answering.

Now that BERT's been added to [TF Hub](https://www.tensorflow.org/hub) as a loadable module, it's easy(ish) to add into existing Tensorflow text pipelines. In an existing pipeline, BERT can replace text embedding layers like ELMO and GloVE. Alternatively, [finetuning](http://wiki.fast.ai/index.php/Fine_tuning) BERT can provide both an accuracy boost and faster training time in many cases.

Here, we'll train a model to predict whether an IMDB movie review is positive or negative using BERT in Tensorflow with tf hub. Some code was adapted from [this colab notebook](https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb). Let's get started!

In [3]:
from sklearn.model_selection import train_test_split
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
from datetime import datetime
from tensorflow import keras
import os
import re
import numpy as np

In addition to the standard libraries we imported above, we'll need to install BERT's python package.

In [4]:
import bert
from bert import run_classifier
from bert import optimization
from bert import tokenization

Below, we'll set an output directory location to store our model output and checkpoints. This can be a local directory, in which case you'd set OUTPUT_DIR to the name of the directory you'd like to create. If you're running this code in Google's hosted Colab, the directory won't persist after the Colab session ends.

Alternatively, if you're a GCP user, you can store output in a GCP bucket. To do that, set a directory name in OUTPUT_DIR and the name of the GCP bucket in the BUCKET field.

Set DO_DELETE to rewrite the OUTPUT_DIR if it exists. Otherwise, Tensorflow will load existing model checkpoints from that directory (if they exist).

In [ ]:
len("Hello it's monday".split())

In [5]:
# Set the output directory for saving model file
# Optionally, set a GCP bucket location

OUTPUT_DIR = 'output-test'#@param {type:"string"}
#@markdown Whether or not to clear/delete the directory and create a new one
DO_DELETE = False #@param {type:"boolean"}
#@markdown Set USE_BUCKET and BUCKET if you want to (optionally) store model output on GCP bucket.
USE_BUCKET = False #@param {type:"boolean"}
BUCKET = 'BUCKET_NAME' #@param {type:"string"}

if USE_BUCKET:
  OUTPUT_DIR = 'gs://{}/{}'.format(BUCKET, OUTPUT_DIR)
  from google.colab import auth
  auth.authenticate_user()

if DO_DELETE:
  try:
    tf.gfile.DeleteRecursively(OUTPUT_DIR)
  except:
    # Doesn't matter if the directory didn't exist
    pass
tf.gfile.MakeDirs(OUTPUT_DIR)
print('***** Model output directory: {} *****'.format(OUTPUT_DIR))


***** Model output directory: output-test *****


#Data

First, let's download the dataset, hosted by Stanford. The code below, which downloads, extracts, and imports the IMDB Large Movie Review Dataset, is borrowed from [this Tensorflow tutorial](https://www.tensorflow.org/hub/tutorials/text_classification_with_tf_hub).

In [6]:
# Load all files from a directory in a DataFrame.
def load_directory_data(directory):
  data = {}
  data["sentence"] = []
  data["sentiment"] = []
  for file_path in os.listdir(directory):
    with tf.gfile.GFile(os.path.join(directory, file_path), "r") as f:
      data["sentence"].append(f.read())
      data["sentiment"].append(re.match("\d+_(\d+)\.txt", file_path).group(1))
  return pd.DataFrame.from_dict(data)

# Merge positive and negative examples, add a polarity column and shuffle.
def load_dataset(directory):
  pos_df = load_directory_data(os.path.join(directory, "pos"))
  neg_df = load_directory_data(os.path.join(directory, "neg"))
  pos_df["polarity"] = 1
  neg_df["polarity"] = 0
  return pd.concat([pos_df, neg_df]).sample(frac=1).reset_index(drop=True)

# Download and process the dataset files.
def download_and_load_datasets(force_download=False):
  dataset = tf.keras.utils.get_file(
      fname="aclImdb.tar.gz", 
      origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
      extract=True)
  
  train_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                       "aclImdb", "train"))
  test_df = load_dataset(os.path.join(os.path.dirname(dataset), 
                                      "aclImdb", "test"))
  
  return train_df, test_df


In [3]:
df_train = pd.DataFrame()
df_train = pd.read_csv("Data/trainIMDB7000.csv", sep =";")
df_test = pd.DataFrame()
df_test = pd.read_csv("Data/testIMDB3000.csv", sep =";")
print(df_train['Tag'].value_counts())
print(df_test['Tag'].value_counts())

1    3750
0    3750
Name: Tag, dtype: int64
1    1250
0    1250
Name: Tag, dtype: int64


In [7]:
def normalize_text(text):
    #s = text.split()
    #s = [t for t in s if (t not in stopwords.words('english')) or (t not in other_stopwords)]
    #s = [t for t in s if t not in other_stopwords]
    #s = " ".join(s)
    no_tabs = text.lower().replace('\t', ' ')
    remove_tag = re.sub(r'@[A-Za-z0-9]+', "", no_tabs)
    remove_url = re.sub(r'https?://[A-Za-z0-9./]+', "", remove_tag)
    alpha_only = re.sub("<br\s*/><br\s*/>", " ", no_tabs)
    #multi_spaces = re.sub("[^a-zA-Z\.\!]", " ", alpha_only) !!!!!! temporary
    multi_spaces = re.sub("[^a-zA-Z\!]", " ", alpha_only)
    text_clean = re.sub(" +", " ", multi_spaces);
    return text_clean

In [8]:
train_init, test_init = download_and_load_datasets()
train_init.drop(["sentiment"], inplace=True, axis=1)
test_init.drop(["sentiment"], inplace=True, axis=1)
train_init.columns = ["Phrase", "Tag"]
test_init.columns = ["Phrase", "Tag"]

In [9]:
print(train_init.shape)
print(train_init.head())

(25000, 2)
                                              Phrase  Tag
0  Zodiac Killer. 1 out of 10. Worst acting ever....    0
1  The film is poorly casted, except for some fam...    0
2  A flying saucer manned (literally) by a crew o...    1
3  Esther Williams gets her first post MGM starri...    1
4  I was always a big fan of this movie, first of...    1


In [ ]:
print(train_init.head())
df_train_clean = train_init.copy()
df_train_clean.Phrase = [normalize_text(row) for row in train_init.Phrase]
print(df_train_clean.head())
df_test_clean = test_init.copy()
df_test_clean.Phrase = [normalize_text(row) for row in df_test_clean.Phrase]

In [4]:
train_init = df_train.reindex(np.random.permutation(df_train.index))
test_init = df_test.reindex(np.random.permutation(df_test.index))

In [10]:
print(test_init.shape)
print(test_init.head())
print(test_init.Phrase[0])
print(train_init.Tag.value_counts())
print(df_test.shape)
print(df_test.head())
print(df_test.Phrase[0])
print(df_train.Tag.value_counts())

(25000, 2)
                                              Phrase  Tag
0  The emergence of Quentin Tarantino and his dub...    0
1  THE PLOT: A trucker (Kristofferson) battles a ...    0
2  Hm. Where do I start? I usually ignore whateve...    0
3  Well another shootem up. Typical run around fi...    0
4  The Polar Express. Director Robert Zemeckis, I...    0
The emergence of Quentin Tarantino and his dubious influence on the likes of Guy Ritchie may have triggered the wave of appalling British gangster flicks we've been bequeathed over the past few years, but one of our most famous acting exports only serves to perpetuate the cycle by lending his considerable name to trash like this. I only wish he'd taken a moment to consider before choosing this project for the same reasons of personal gain he admits he often employs. It's not only stifling HIS talent, but possibly the promise of future originality from British films. <br /><br />Not one of this film's characters are likeable or even r

NameError: name 'df_test' is not defined

To keep training fast, we'll take a sample of 5000 train and test examples, respectively.

In [11]:
train = train_init#.sample(10000)
test = test_init#.sample(3000)

In [ ]:
train.to_csv('trainIMDBkeras7000.csv',sep=";")
test.to_csv('testIMDBkeras3000.csv', sep=";")

In [9]:
train.columns
print(train.Tag.value_counts())
print(test.shape)
print(test.head())
#print(test['Phrase'][0])
print(test.Tag.value_counts())

1    5036
0    4964
Name: Tag, dtype: int64
(3000, 2)
                                                  Phrase  Tag
23318  Mockney comes to Brighton; despite a poor rece...    0
9812   Oh yes, I admit I have made myself guilty of t...    0
3932   "Wagons East" was a big disappointment for me....    0
18527  I do not think I am alone when I say that 2005...    1
21097  This has got to be the most god-awful piece of...    0
0    1506
1    1494
Name: Tag, dtype: int64


For us, our input data is the 'sentence' column and our label is the 'polarity' column (0, 1 for negative and positive, respecitvely)

In [12]:
DATA_COLUMN = 'Phrase'
LABEL_COLUMN = 'Tag'
# label_list is the list of labels, i.e. True, False or 0, 1 or 'dog', 'cat'
label_list = [0, 1]

#Data Preprocessing
We'll need to transform our data into a format BERT understands. This involves two steps. First, we create  `InputExample`'s using the constructor provided in the BERT library.

- `text_a` is the text we want to classify, which in this case, is the `Request` field in our Dataframe. 
- `text_b` is used if we're training a model to understand the relationship between sentences (i.e. is `text_b` a translation of `text_a`? Is `text_b` an answer to the question asked by `text_a`?). This doesn't apply to our task, so we can leave `text_b` blank.
- `label` is the label for our example, i.e. True, False

In [13]:
# Use the InputExample class from BERT's run_classifier code to create examples from the data
train_InputExamples = train.apply(lambda x: bert.run_classifier.InputExample(guid=None, # Globally unique ID for bookkeeping, unused in this example
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

test_InputExamples = test.apply(lambda x: bert.run_classifier.InputExample(guid=None, 
                                                                   text_a = x[DATA_COLUMN], 
                                                                   text_b = None, 
                                                                   label = x[LABEL_COLUMN]), axis = 1)

Next, we need to preprocess our data so that it matches the data BERT was trained on. For this, we'll need to do a couple of things (but don't worry--this is also included in the Python library):


1. Lowercase our text (if we're using a BERT lowercase model)
2. Tokenize it (i.e. "sally says hi" -> ["sally", "says", "hi"])
3. Break words into WordPieces (i.e. "calling" -> ["call", "##ing"])
4. Map our words to indexes using a vocab file that BERT provides
5. Add special "CLS" and "SEP" tokens (see the [readme](https://github.com/google-research/bert))
6. Append "index" and "segment" tokens to each input (see the [BERT paper](https://arxiv.org/pdf/1810.04805.pdf))

Happily, we don't have to worry about most of these details.




To start, we'll need to load a vocabulary file and lowercasing information directly from the BERT tf hub module:

In [14]:
# This is a path to an uncased (all lowercase) version of BERT
BERT_MODEL_HUB = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

def create_tokenizer_from_hub_module():
  """Get the vocab file and casing info from the Hub module."""
  with tf.Graph().as_default():
    bert_module = hub.Module(BERT_MODEL_HUB)
    tokenization_info = bert_module(signature="tokenization_info", as_dict=True)
    with tf.Session() as sess:
      vocab_file, do_lower_case = sess.run([tokenization_info["vocab_file"],
                                            tokenization_info["do_lower_case"]])
      
  return bert.tokenization.FullTokenizer(
      vocab_file=vocab_file, do_lower_case=do_lower_case)

tokenizer = create_tokenizer_from_hub_module()

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Downloading TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.
INFO:tensorflow:Downloading https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1: 281.62MB
INFO:tensorflow:Downloaded https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1, Total size: 421.86MB
INFO:tensorflow:Downloaded TF-Hub Module 'https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1'.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


Great--we just learned that the BERT model we're using expects lowercase data (that's what stored in tokenization_info["do_lower_case"]) and we also loaded BERT's vocab file. We also created a tokenizer, which breaks words into word pieces:

In [ ]:
tokenizer.tokenize("This here's an example of using the BERT tokenizer")

Using our tokenizer, we'll call `run_classifier.convert_examples_to_features` on our InputExamples to convert them into features BERT understands.

In [15]:
# We'll set sequences to be at most 128 tokens long.
MAX_SEQ_LENGTH = 128
# Convert our train and test features to InputFeatures that BERT understands.
train_features = bert.run_classifier.convert_examples_to_features(train_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)
test_features = bert.run_classifier.convert_examples_to_features(test_InputExamples, label_list, MAX_SEQ_LENGTH, tokenizer)

INFO:tensorflow:Writing example 0 of 25000
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] zodiac killer . 1 out of 10 . worst acting ever . no really worst acting ever . david hess ( last house on the left . no the one from the seventies . rent it it ' s really good ) is the worst of the bunch ( pretty stiff competition but he is amazingly god - awful . ) one would be hard pressed to find a home movie participant with such an awkward camera presence . the film actually sc ##ree ##ches to a stunning painful halt when he is on the screen . < br / > < br / > not that the film actually has any red ##eem ##ing qualities for mr . hess to ruin . it is filmed [SEP]
INFO:tensorflow:input_ids: 101 28501 6359 1012 1015 2041 1997 2184 1012 5409 3772 2412 1012 2053 2428 5409 3772 2412 1012 2585 23484 1006 2197 2160 2006 1996 2187 1012 2053 1996 2028 2013 1996 26232 1012 9278 2009 2009 1005 1055 2428 2204 1007 2003 1996 5409 1997 1996 9129 1006 3492 10551 29

INFO:tensorflow:input_ids: 101 1045 2001 2467 1037 2502 5470 1997 2023 3185 1010 2034 1997 2035 2031 2017 2464 1996 3459 1010 1996 3772 2003 21688 1998 2393 2191 2023 3185 2693 2247 2200 2092 1012 22330 24457 11133 2001 2445 2307 4391 2005 2014 2535 1010 1998 2027 2020 2092 10849 1012 1996 2927 1997 2023 3185 4627 1999 1996 2627 2043 26879 5076 5134 1006 22330 24457 1007 3005 3861 1011 3819 3510 3310 2000 1037 21797 9190 2043 2014 3129 17438 8289 14153 1012 14599 1010 10223 3366 4152 1037 2117 2915 2012 2166 2043 2002 10217 2000 2022 1000 22207 1000 2067 2000 3011 2004 1996 20662 4074 16133 1006 2728 2091 3240 1010 3781 1007 1012 4074 3632 2006 2000 2444 2010 2047 2166 102
INFO:tensorflow:input_mask: 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
INFO:tensorflow:segment_ids: 0 0 0

INFO:tensorflow:segment_ids: 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
INFO:tensorflow:label: 0 (id = 0)
INFO:tensorflow:*** Example ***
INFO:tensorflow:guid: None
INFO:tensorflow:tokens: [CLS] the polar express . director robert ze ##me ##cki ##s , i love back to the future , forrest gum ##p , contact , and who framed roger rabbit ( no question mark after that movie title ! ! ) . and tom hank ##s , one of my favorite actors . the reviews of this movie were almost unanimous saying that this is an instant holiday classic . ebert & roe ##per give it two enthusiastic thumbs up ! even ebert ' s written review gave it a full four stars ! wow . . . ok . . . this i gotta see ! but wait . . . the motion capture used looks really weird . hmm . . . maybe i ' [SEP]
INFO:tensorflow:input

#Creating a model

Now that we've prepared our data, let's focus on building a model. `create_model` does just this below. First, it loads the BERT tf hub module again (this time to extract the computation graph). Next, it creates a single new layer that will be trained to adapt BERT to our sentiment task (i.e. classifying whether a movie review is positive or negative). This strategy of using a mostly trained model is called [fine-tuning](http://wiki.fast.ai/index.php/Fine_tuning).

In [16]:
def create_model(is_predicting, input_ids, input_mask, segment_ids, labels,
                 num_labels):
  """Creates a classification model."""

  bert_module = hub.Module(
      BERT_MODEL_HUB,
      trainable=True)
  bert_inputs = dict(
      input_ids=input_ids,
      input_mask=input_mask,
      segment_ids=segment_ids)
  bert_outputs = bert_module(
      inputs=bert_inputs,
      signature="tokens",
      as_dict=True)

  # Use "pooled_output" for classification tasks on an entire sentence.
  # Use "sequence_outputs" for token-level output.
  output_layer = bert_outputs["pooled_output"]

  hidden_size = output_layer.shape[-1].value

  # Create our own layer to tune for politeness data.
  output_weights = tf.get_variable(
      "output_weights", [num_labels, hidden_size],
      initializer=tf.truncated_normal_initializer(stddev=0.02))

  output_bias = tf.get_variable(
      "output_bias", [num_labels], initializer=tf.zeros_initializer())

  with tf.variable_scope("loss"):

    # Dropout helps prevent overfitting
    output_layer = tf.nn.dropout(output_layer, keep_prob=0.9)

    logits = tf.matmul(output_layer, output_weights, transpose_b=True)
    logits = tf.nn.bias_add(logits, output_bias)
    log_probs = tf.nn.log_softmax(logits, axis=-1)

    # Convert labels into one-hot encoding
    one_hot_labels = tf.one_hot(labels, depth=num_labels, dtype=tf.float32)

    predicted_labels = tf.squeeze(tf.argmax(log_probs, axis=-1, output_type=tf.int32))
    # If we're predicting, we want predicted labels and the probabiltiies.
    if is_predicting:
      return (predicted_labels, log_probs)

    # If we're train/eval, compute loss between predicted and actual label
    per_example_loss = -tf.reduce_sum(one_hot_labels * log_probs, axis=-1)
    loss = tf.reduce_mean(per_example_loss)
    return (loss, predicted_labels, log_probs)


Next we'll wrap our model function in a `model_fn_builder` function that adapts our model to work for training, evaluation, and prediction.

In [17]:
# model_fn_builder actually creates our model function
# using the passed parameters for num_labels, learning_rate, etc.
def model_fn_builder(num_labels, learning_rate, num_train_steps,
                     num_warmup_steps):
  """Returns `model_fn` closure for TPUEstimator."""
  def model_fn(features, labels, mode, params):  # pylint: disable=unused-argument
    """The `model_fn` for TPUEstimator."""

    input_ids = features["input_ids"]
    input_mask = features["input_mask"]
    segment_ids = features["segment_ids"]
    label_ids = features["label_ids"]

    is_predicting = (mode == tf.estimator.ModeKeys.PREDICT)
    
    # TRAIN and EVAL
    if not is_predicting:

      (loss, predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      train_op = bert.optimization.create_optimizer(
          loss, learning_rate, num_train_steps, num_warmup_steps, use_tpu=False)

      # Calculate evaluation metrics. 
      def metric_fn(label_ids, predicted_labels):
        accuracy = tf.metrics.accuracy(label_ids, predicted_labels)
        f1_score = tf.contrib.metrics.f1_score(
            label_ids,
            predicted_labels)
        auc = tf.metrics.auc(
            label_ids,
            predicted_labels)
        recall = tf.metrics.recall(
            label_ids,
            predicted_labels)
        precision = tf.metrics.precision(
            label_ids,
            predicted_labels) 
        true_pos = tf.metrics.true_positives(
            label_ids,
            predicted_labels)
        true_neg = tf.metrics.true_negatives(
            label_ids,
            predicted_labels)   
        false_pos = tf.metrics.false_positives(
            label_ids,
            predicted_labels)  
        false_neg = tf.metrics.false_negatives(
            label_ids,
            predicted_labels)
        return {
            "eval_accuracy": accuracy,
            "f1_score": f1_score,
            "auc": auc,
            "precision": precision,
            "recall": recall,
            "true_positives": true_pos,
            "true_negatives": true_neg,
            "false_positives": false_pos,
            "false_negatives": false_neg
        }

      eval_metrics = metric_fn(label_ids, predicted_labels)

      if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode=mode,
          loss=loss,
          train_op=train_op)
      else:
          return tf.estimator.EstimatorSpec(mode=mode,
            loss=loss,
            eval_metric_ops=eval_metrics)
    else:
      (predicted_labels, log_probs) = create_model(
        is_predicting, input_ids, input_mask, segment_ids, label_ids, num_labels)

      predictions = {
          'probabilities': log_probs,
          'labels': predicted_labels
      }
      return tf.estimator.EstimatorSpec(mode, predictions=predictions)

  # Return the actual model function in the closure
  return model_fn


In [18]:
# Compute train and warmup steps from batch size
# These hyperparameters are copied from this colab notebook (https://colab.sandbox.google.com/github/tensorflow/tpu/blob/master/tools/colab/bert_finetuning_with_cloud_tpus.ipynb)
BATCH_SIZE = 32
LEARNING_RATE = 2e-5
NUM_TRAIN_EPOCHS = 3.0
# Warmup is a period of time where hte learning rate 
# is small and gradually increases--usually helps training.
WARMUP_PROPORTION = 0.1
# Model configs
SAVE_CHECKPOINTS_STEPS = 500
SAVE_SUMMARY_STEPS = 100

In [19]:
# Compute # train and warmup steps from batch size
num_train_steps = int(len(train_features) / BATCH_SIZE * NUM_TRAIN_EPOCHS)
num_warmup_steps = int(num_train_steps * WARMUP_PROPORTION)

In [20]:
# Specify output directory and number of checkpoint steps to save
run_config = tf.estimator.RunConfig(
    model_dir=OUTPUT_DIR,
    save_summary_steps=SAVE_SUMMARY_STEPS,
    save_checkpoints_steps=SAVE_CHECKPOINTS_STEPS)

In [21]:
model_fn = model_fn_builder(
  num_labels=len(label_list),
  learning_rate=LEARNING_RATE,
  num_train_steps=num_train_steps,
  num_warmup_steps=num_warmup_steps)

estimator = tf.estimator.Estimator(
  model_fn=model_fn,
  config=run_config,
  params={"batch_size": BATCH_SIZE})


INFO:tensorflow:Using config: {'_model_dir': 'output-test', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': 500, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f520b58f3c8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


Next we create an input builder function that takes our training feature set (`train_features`) and produces a generator. This is a pretty standard design pattern for working with Tensorflow [Estimators](https://www.tensorflow.org/guide/estimators).

In [22]:
# Create an input function for training. drop_remainder = True for using TPUs.
train_input_fn = bert.run_classifier.input_fn_builder(
    features=train_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=True,
    drop_remainder=False)

Now we train our model! For me, using a Colab notebook running on Google's GPUs, my training time was about 14 minutes.

In [23]:
print(f'Beginning Training!')
current_time = datetime.now()
estimator.train(input_fn=train_input_fn, max_steps=num_train_steps)
print("Training took time ", datetime.now() - current_time)

Beginning Training!
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


/home/supercalculateur/environment/default_gpu/lib/python3.6/site-packages/tensorflow/python/ops/gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output-test/model.ckpt-0
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into output-test/model.ckpt.
INFO:tensorflow:loss = 0.73726887, step = 0
INFO:tensorflow:global_step/sec: 1.9122
INFO:tensorflow:loss = 0.5044448, step = 100 (52.296 sec)
INFO:tensorflow:global_step/sec: 2.37167
INFO:tensorflow:loss = 0.13499644, step = 200 (42.164 sec)
INFO:tensorflow:global_step/sec: 2.36522
INFO:tensorflow:loss = 0.51838756, step = 300 (42.279 sec)
INFO:tensorflow:global_step/sec: 2.36444
INFO:tensorflow:loss = 0.46407074, step = 400 (42.293 sec)
INFO:tensorflow:Saving checkpoints for 500 into output-test/model.ckpt.
INFO:tensorflow:global_step/sec: 2.25737
INFO:tensorflow:loss = 0.20553747, step = 500 (44.299 sec)
INFO:tensorflow:global_step/sec: 2.36169
INFO:

Now let's use our test data to see how well our model did:

In [24]:
test_input_fn = run_classifier.input_fn_builder(
    features=test_features,
    seq_length=MAX_SEQ_LENGTH,
    is_training=False,
    drop_remainder=False)

In [25]:
estimator.evaluate(input_fn=test_input_fn, steps=None)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-04-05-08:18:26
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from output-test/model.ckpt-2343
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-04-05-08:20:10
INFO:tensorflow:Saving dict for global step 2343: auc = 0.88587993, eval_accuracy = 0.88588, f1_score = 0.88476914, false_negatives = 1547.0, false_positives = 1306.0, global_step = 2343, loss = 0.47305262, precision = 0.893466, recall = 0.87624, true_negatives = 11194.0, true_positives = 10953.0
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 2343: output-test/model.ckpt-2343


{'auc': 0.88587993,
 'eval_accuracy': 0.88588,
 'f1_score': 0.88476914,
 'false_negatives': 1547.0,
 'false_positives': 1306.0,
 'loss': 0.47305262,
 'precision': 0.893466,
 'recall': 0.87624,
 'true_negatives': 11194.0,
 'true_positives': 10953.0,
 'global_step': 2343}

Now let's write code to make predictions on new sentences:

In [ ]:
import math
def getPrediction(in_sentences):
    #labels = ["Negative", "Positive"]
  labels = [0, 1]
  input_examples = [run_classifier.InputExample(guid="", text_a = x, text_b = None, label = 0) for x in in_sentences] # here, "" is just a dummy label
  input_features = run_classifier.convert_examples_to_features(input_examples, label_list, MAX_SEQ_LENGTH, tokenizer)
  predict_input_fn = run_classifier.input_fn_builder(features=input_features, seq_length=MAX_SEQ_LENGTH, is_training=False, drop_remainder=False)
  predictions = estimator.predict(predict_input_fn)
  return [(sentence, np.exp(prediction['probabilities']), labels[prediction['labels']]) for sentence, prediction in zip(in_sentences, predictions)]

In [ ]:
pred_sentences = [
  "That movie was absolutely awful",
  "The acting was a bit lacking",
  "The film was creative and surprising",
  "Absolutely fantastic!",
  "I am happy",
  "I am happy!!!",
   ]

In [ ]:
predictions = getPrediction(pred_sentences)

In [ ]:
false_pred = []
for s, prob, pred in predictions:
    if pred != test_init["Phrase"=s].Tag:
        false_pred.append([s, pred, prob])
print(false_pred)

Voila! We have a sentiment classifier!

In [ ]:
predictions